In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import sklearn.gaussian_process.kernels as kl

from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import RBFInterpolator
from scipy.interpolate import NearestNDInterpolator
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA


import em 
import models

# Inductor Guard Ring (GR) 0.35um 


<img src="../img/ind.png" alt="drawing" width="200"/> 


Octagonal inductor in AMS 350 um.


Before running this notebook make sure to [prepare](data_prepare_tmtt_indGR.ipynb) the data.


In [3]:
#model definition
class PreProcess():
    def __init__(self, model, whitening = False, standardize = False) -> None:
        self.model = model
        self.x_scaler = StandardScaler() if standardize else None
        self.pca = PCA(whiten=True) if whitening else None
    
    def fit(self, x_train, y_train):
        if self.pca:
            self.x_mean = np.mean(x_train, axis=0)
            x_train = x_train - self.x_mean
            x_train = self.pca.fit_transform(x_train)
        elif self.x_scaler:
            x_train = self.x_scaler.fit_transform(x_train)

        self.model.fit(x_train, y_train)

    def predict(self, x):
        if self.pca:
            x = x - self.x_mean
            x = self.pca.transform(x)
        elif self.x_scaler:
            x = self.x_scaler.transform(x)
        
        return self.model.predict(x)

class RBFInterpolatorModel:
    def __init__(self, degree=1) -> None:
        self.degree = degree
        
    def fit(self, x_train, y_train):
        self.model = RBFInterpolator(x_train, y_train, 
                     degree=self.degree, smoothing=0.00000001, neighbors=4000)

    def predict(self, x):
        return self.model(x)

class GaussianProcessRegressorModel:
    def __init__(self, kernel) -> None:
        self.model = GaussianProcessRegressor(kernel=kernel, alpha=1e-8)
        
    def fit(self, x_train, y_train):
        self.model.fit(x_train, y_train)

    def predict(self, x):
        return self.model.predict(x)


class NearestNDInterpolatorModel:
    def fit(self, x_train, y_train):
        self.model = NearestNDInterpolator(x_train, y_train)

    def predict(self, x_test):
        return self.model(x_test)

class ANNModel:
    def __init__(self, layers = None, degree = 5) -> None:
       
        self.poly     = PolynomialFeatures(degree)
        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        self.layers = layers if layers else [256, 512]

    def fit(self, x_train, y_train):
        x_t = self.poly.fit_transform(x_train)  
        x_t = self.x_scaler.fit_transform(x_t)

        y_t = self.y_scaler.fit_transform(y_train)

        activation = 'relu'
        output='linear'
        epochs = 1000
        loss="mse" 
        optimizer = keras.optimizers.Adam(learning_rate=0.0001)


        inputs = keras.Input(shape=(x_t.shape[1],), name='parameters')
        
        lay = inputs

        for n in self.layers:
            lay = keras.layers.Dense(n, activation=activation, 
               kernel_regularizer=keras.regularizers.L2(0.000001), 
               activity_regularizer=keras.regularizers.L2(0.001))(lay)

        outputs = keras.layers.Dense(y_t.shape[1], activation=output, 
            kernel_regularizer=keras.regularizers.L2(0.000001))(lay)
        
        self.model = keras.Model(inputs=inputs, outputs=outputs)
        self.model.compile(
            loss=loss,
            optimizer=optimizer)
        
        self.history = self.model.fit(x_t, y_t, 
                    epochs = epochs, 
                    batch_size= 64, 
                    verbose = 0)


    def predict(self, x_test):
        return self.y_scaler.inverse_transform(
            self.model.predict(self.x_scaler.transform(
                self.poly.transform(x_test))))

In [4]:
def model_atf():
    _list = []
    _list.append(PreProcess(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)), standardize=True))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=7), whitening=True))

    _list.append(PreProcess(RandomForestRegressor(max_depth= 15, max_samples=0.8), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [256, 512]))

    
    return _list

def model_at():
    _list = []
    _list.append(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=6), whitening=True))
    
    _list.append(PreProcess(RandomForestRegressor(), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [512, 1024]))
    
    return _list

def model_list():
    _list = []

    _list.append(PreProcess(GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5)), standardize=True))

    _list.append(KernelRidge(alpha = 0.00000000001, 
          kernel=1e-5 * kl.RationalQuadratic(length_scale=1e-5, alpha=1e-5) + 1e-5**1*kl.RBF(length_scale=30)))

    _list.append(PreProcess(RBFInterpolatorModel(degree=4), whitening=True))

    _list.append(PreProcess(RandomForestRegressor(), whitening=True))
    
    _list.append(NearestNDInterpolatorModel())
    
    _list.append(ANNModel(layers = [256, 512]))

    
    return _list


def srf_models():
    _list = []
    _list.append(PreProcess(RBFInterpolatorModel(degree=2), whitening=True))
    _list.append(PreProcess(RBFInterpolatorModel(degree=3), whitening=True))
    return _list


mdls_atf = model_atf()
mdls_at = model_at()

def ranges(df_train, df_test):
    df_test = df_test[df_test.Din > 70]
    df_test = df_test[df_test.W > 6]
    df_test = df_test[df_test.W < 14]
    df_test = df_test[df_test.Din < 180 ]
    
    return  df_train, df_test

data_folder = "../data/inductGR_350nm_200GHz/"

In [5]:
# freq and turn dependent model
np.random.seed(1234)
tf.random.set_seed(1234)
errors_atf = []

freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder)
freq, _, __, x_test, y_test, _ = models.load_data(data_folder, 6)
x_test = np.c_[ [5.00075]*len(x_test), x_test]


hello package
hello package


In [6]:
errors_atf.append(models.test_it(mdls_atf, freq, x_train, y_train, x_test, y_test))

print("ATF")
models.error_to_csv(errors_atf)


2022-05-19 14:38:39.508143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-19 14:38:39.508163: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-19 14:38:39.508181: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cad62.icg): /proc/driver/nvidia/version does not exist
2022-05-19 14:38:39.508313: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
# freq and turn dependent model
np.random.seed(1234)
tf.random.set_seed(1234)
errors_atf = []

freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder)
freq, _, __, x_test, y_test, _ = models.load_data(data_folder, 6, filter=ranges)
x_test = np.c_[ [5.00075]*len(x_test), x_test]
errors_atf.append(models.test_it(mdls_atf, freq, x_train, y_train, x_test, y_test))

print("ATF")
models.error_to_csv(errors_atf)

ATF
L, None, None, None, None, 919.94, 52.41, 262.90, 6.43, 188.35, 6.11, 281.80, 5.85, 
Q, None, None, None, None, 4503.30, 123.91, 332.17, 9.98, 188.36, 8.03, 335.95, 14.08, 


In [29]:
# trun dependent model
np.random.seed(1234)
tf.random.set_seed(1234)
errors_at = []

freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder, 6, filter=ranges)
errors_at.append(models.test_it(model_list(), freq, x_train, y_train, x_test, y_test))
print("AT")
models.error_to_csv(errors_at)

/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


AT
L, 100.00, 100.00, 717.16, 25.66, 10.16, 0.22, 346.95, 6.59, 188.35, 6.11, 19.02, 1.10, 
Q, 100.00, 100.00, 10305.60, 225.86, 10.17, 0.77, 388.21, 8.29, 188.36, 8.03, 24.21, 4.87, 


In [36]:
freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder, 6, filter=ranges)
mdls_at[0] = GaussianProcessRegressorModel(0.1**2 * kl.RationalQuadratic(length_scale=1e-8, alpha=1e-8) + 1**2*kl.RBF(length_scale=5))

In [31]:
#GPR  AllT
mdls_at[0].fit(x_train, y_train)
pred = mdls_at[0].predict(x_test)
errors_at[0][0] = em.mape_lq_diff(freq, pred, y_test)

In [37]:
mdls_at[1] = KernelRidge(alpha = 0.00000000001, 
          kernel=0.1**2*kl.RBF() + 0.5**2*kl.RBF() + 0.25**2*kl.RBF() + 1e-5 * kl.RationalQuadratic())
mdls_at[1].fit(x_train, y_train)
pred = mdls_at[1].predict(x_test)
errors_at[0][1] = em.mape_lq_diff(freq, pred, y_test)

In [38]:
print("AT")
models.error_to_csv(errors_at)

AT
L, 369.49, 13.26, 102.72, 54.82, 10.16, 0.22, 346.95, 6.59, 188.35, 6.11, 19.02, 1.10, 
Q, 277.15, 53.34, 113.93, 95.59, 10.17, 0.77, 388.21, 8.29, 188.36, 8.03, 24.21, 4.87, 


In [11]:
#turn and freq independent @ 5GHz
np.random.seed(1234)
tf.random.set_seed(1234)
errors = []

for nt in range(1, 6):
    freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder, 6, nt = nt)
    errors.append(models.test_it(model_list(), freq, x_train, y_train, x_test, y_test))

print("No SRF Filter")
models.error_to_csv(errors, line_headers = [f'N={nt}' for nt in range(1, 6)])

/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


No SRF Filter
N=1, L, 0.17, 0.05, 0.49, 0.17, 0.16, 0.05, 4.14, 1.36, 5.62, 1.14, 0.53, 0.16, 
N=1, Q, 0.77, 0.21, 5.48, 1.88, 0.57, 0.15, 12.33, 3.69, 18.65, 5.59, 2.48, 0.57, 
N=2, L, 0.12, 0.03, 0.50, 0.19, 0.11, 0.03, 4.65, 2.12, 6.33, 1.35, 0.88, 0.23, 
N=2, Q, 0.32, 0.12, 3.37, 1.54, 0.23, 0.06, 9.94, 3.43, 15.75, 4.83, 4.27, 0.71, 
N=3, L, 0.09, 0.03, 0.53, 0.20, 0.18, 0.04, 5.95, 2.22, 5.59, 1.41, 0.96, 0.26, 
N=3, Q, 0.31, 0.10, 2.75, 1.16, 0.32, 0.05, 12.57, 3.79, 10.68, 2.84, 8.81, 1.55, 
N=4, L, 0.10, 0.03, 0.82, 0.29, 0.15, 0.05, 8.43, 2.81, 8.65, 2.76, 1.97, 0.50, 
N=4, Q, 0.42, 0.10, 5.67, 0.72, 0.61, 0.12, 15.58, 4.29, 13.88, 3.38, 7.39, 1.73, 
N=5, L, 29.24, 1.55, 304.56, 13.03, 39.91, 1.96, 2097.18, 101.42, 2366.15, 112.67, 281.16, 14.37, 
N=5, Q, 28.54, 1.59, 306.66, 14.39, 40.72, 2.11, 2679.41, 137.80, 2794.11, 173.28, 277.70, 16.00, 


In [12]:
#turn and freq independent @ 5GHz SRF > 6GHz
np.random.seed(1234)
tf.random.set_seed(1234)

errors = []

for nt in range(1, 6):
    freq, x_train, y_train, x_test, y_test, _ = models.load_data(data_folder, 6, nt = nt, srf=7.0)
    errors.append(models.test_it(model_list(), freq, x_train, y_train, x_test, y_test))

print("SRF Filter")
models.error_to_csv(errors, line_headers = [f'N={nt}' for nt in range(1, 6)])




/home/nlourenco/.pyenv/.py38env/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


SRF Filter
N=1, L, 0.17, 0.05, 0.49, 0.17, 0.16, 0.05, 4.14, 1.36, 5.62, 1.14, 0.53, 0.16, 
N=1, Q, 0.77, 0.21, 5.48, 1.88, 0.57, 0.15, 12.33, 3.69, 18.65, 5.59, 2.48, 0.57, 
N=2, L, 0.12, 0.03, 0.50, 0.19, 0.11, 0.03, 4.65, 2.12, 6.33, 1.35, 0.88, 0.23, 
N=2, Q, 0.32, 0.12, 3.37, 1.54, 0.23, 0.06, 9.94, 3.43, 15.75, 4.83, 4.27, 0.71, 
N=3, L, 0.09, 0.03, 0.53, 0.20, 0.18, 0.04, 5.95, 2.22, 5.59, 1.41, 0.96, 0.26, 
N=3, Q, 0.31, 0.10, 2.75, 1.16, 0.32, 0.05, 12.57, 3.79, 10.68, 2.84, 8.81, 1.55, 
N=4, L, 0.10, 0.03, 0.84, 0.29, 0.16, 0.04, 6.08, 2.39, 5.09, 2.07, 2.11, 0.55, 
N=4, Q, 0.34, 0.08, 3.24, 0.85, 0.28, 0.08, 7.76, 3.01, 8.81, 2.26, 4.53, 1.53, 
N=5, L, 0.09, 0.04, 1.88, 0.50, 0.08, 0.05, 17.80, 6.55, 14.57, 5.24, 5.20, 0.89, 
N=5, Q, 0.17, 0.08, 8.02, 2.06, 0.21, 0.06, 23.31, 5.23, 18.05, 4.50, 38.95, 4.99, 


In [13]:
# SRF Filter at + 10GHz
np.random.seed(1234)
tf.random.set_seed(1234)
srf_errors = []

for nt in range(1, 6):
    print(f"Subspace: {nt}")
    freq, x_train, y_train, x_test, y_test, srf_data = models.load_data(data_folder, 6, nt=nt, n_samples=1000)    
    srf_errors.append(models.test_it_srf(srf_models(), srf_data))

print("SRF filter error")
models.error_to_csv(srf_errors, line_headers = [f'N={nt}' for nt in range(1, 6)], error_headers = ['SRF, '])


Subspace: 1
Subspace: 2
Subspace: 3
Subspace: 4
Subspace: 5
SRF filter error
N=1, SRF, 922.38, 563.74, 921.84, 563.76, 
N=2, SRF, 221.39, 76.67, 223.09, 76.64, 
N=3, SRF, 52.59, 8.00, 52.46, 7.98, 
N=4, SRF, 9.05, 3.15, 9.03, 3.46, 
N=5, SRF, 12.81, 3.49, 12.81, 3.48, 


In [14]:
#turn and freq independent @ 5GHz SRF > 7GHz
import pickle
import em
import importlib

importlib.reload(models)

np.random.seed(1234)
tf.random.set_seed(1234)

model = {}
for nt in range(1, 6):
    for f in range(204):
        freq, x_train, y_train, x_test, y_test, srf_data = models.load_data(data_folder, f, nt = nt, srf=7.0, skip_test=True)
        m = models.RBFInterpolatorModel(degree=4)
        m.fit(x_train, y_train)
        mdl = {
            'freq_index': f,
            'freq': freq[0],
            'nt': nt,
            'RBFmodel': m
        }

        model[(nt, f)] = mdl
    
    srf_m = models.RBFInterpolatorModel(degree=2)
    srf_m.fit(srf_data[0], srf_data[1])        
    srf_mdl = {
            'srf_limit': 7,
            'nt': nt,
            'RBFmodel': srf_m
    }
    model[(nt, 'srf')] = srf_mdl


model['inputs'] = ['Din', 'W']
model['ouputs'] = em.COLUMNS_2P[1:]
model['ranges'] = {'din': (70, 180), 'w': (6, 14), 'nt': (1,5), 'freq': (0, 200)}
model['device'] = 'ind'
model['key'] = {'nt': [1, 2, 3, 4 ,5], 'f':(0, 203)}

with open("../PASSIVES_RBF_IND_5G_SRF7_200GHz_tmtt.model",'wb') as outfile:
    pickle.dump(model, outfile)    

###### Copyright (C) 2022 Instituto de Telecomunicações & IMSE CSIC